In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

# Specify the source file path in your Colab environment
source_file_path = '/content/drive/MyDrive/DataChallenge/dataset_1.zip'

# Specify the destination file path in your Google Drive
destination_file_path = '/content/dataset_1.zip'

# Copy the file from the source path to the destination path
shutil.copyfile(source_file_path, destination_file_path)

'/content/dataset_1.zip'

In [ ]:
!unzip dataset_1.zip

Archive:  dataset_1.zip
   creating: dataset_1/
   creating: dataset_1/OK/
  inflating: dataset_1/OK/CH214FC2_342055_0_RESPONSIVITE_3.png  
  inflating: dataset_1/OK/CH214FC2_342060_0_RESPONSIVITE_3.png  
  inflating: dataset_1/OK/CH214FC2_342064_0_RESPONSIVITE_3.png  
  inflating: dataset_1/OK/CH214FC2_342077_0_RESPONSIVITE_3.png  
  inflating: dataset_1/OK/CH214FC2_342171_0_RESPONSIVITE_3.png  
  inflating: dataset_1/OK/CH214FC2_342173_0_RESPONSIVITE_3.png  
  inflating: dataset_1/OK/CH214FC2_342175_0_RESPONSIVITE_3.png  
  inflating: dataset_1/OK/CH214FC2_342177_0_RESPONSIVITE_3.png  
  inflating: dataset_1/OK/CH214FC2_342179_0_RESPONSIVITE_3.png  
  inflating: dataset_1/OK/CH214FC2_342182_0_RESPONSIVITE_3.png  
  inflating: dataset_1/OK/CH214FC2_342186_0_RESPONSIVITE_3.png  
  inflating: dataset_1/OK/CH214FC2_342188_0_RESPONSIVITE_3.png  
  inflating: dataset_1/OK/CH214FC2_342190_0_RESPONSIVITE_3.png  
  inflating: dataset_1/OK/CH214FC2_342194_0_RESPONSIVITE_3.png  
  inflating: da

In [ ]:
# Importing libraries
import numpy as np
import os
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import ResNet101, ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import Sequential, layers
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.layers import Rescaling
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Lambda


In [ ]:
# Specifications
batch_size = 16
img_height = 512
img_width = 640

#Load dataset
file_dir = pathlib.Path('/content/dataset_1')


train_ds = tf.keras.utils.image_dataset_from_directory(
  file_dir,
  validation_split=0.2,
  subset="training",
  image_size= (512, 640),
  label_mode = 'categorical',
  color_mode = "grayscale",
  seed = 457,
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  file_dir,
  validation_split=0.2,
  image_size= (512, 640),
  subset="validation",
  label_mode = 'categorical',
  seed = 457,
  color_mode = "grayscale",
  batch_size=batch_size)


Found 200 files belonging to 2 classes.
Using 160 files for training.
Found 200 files belonging to 2 classes.
Using 40 files for validation.


In [ ]:
# Get the file names from the training dataset
train_filenames = [str(pathlib.Path(path).name) for path in train_ds.file_paths]

# Get the file names from the validation dataset
val_filenames = [str(pathlib.Path(path).name) for path in val_ds.file_paths]

# Convert the file names to sets
train_set = set(train_filenames)
val_set = set(val_filenames)

# Find the intersection of the two sets
common_images = train_set.intersection(val_set)

# Check if the intersection is empty
if len(common_images) == 0:
    print("No common images between training and validation datasets.")
else:
    print("Common images found between training and validation datasets:", common_images)

No common images between training and validation datasets.


In [ ]:
# # Names of the classes and Visualization of the data
# class_names = train_generator.class_names
# print(class_names)

AttributeError: ignored

In [ ]:
# Load the pre-trained ResNet50 model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the pre-trained layers
for layer in resnet.layers:
    layer.trainable = False

# Convert grayscale to RGB
def grayscale_to_rgb(image):
    return tf.image.grayscale_to_rgb(image)

# Add new layers for your own dataset
inputs = tf.keras.Input(shape=(img_height, img_width, 1))
rgb_images = Lambda(grayscale_to_rgb)(inputs)
x = resnet(rgb_images)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.25)(x)
predictions = Dense(2, activation='softmax')(x)

# Create a new model
model = Model(inputs=inputs, outputs=predictions)
opt = tf.keras.optimizers.experimental.AdamW()

def scheduler(epoch, lr):
   if epoch < 5:
     return lr
   else:
     return lr * tf.math.exp(-0.1)
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
# Compile the model
model.compile(optimizer= 'adam', loss='categorical_crossentropy', 
              metrics=['accuracy'])




# Define the checkpoint path and format
checkpoint_path = "model_checkpoint_{epoch:02d}.h5"

# Create the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(checkpoint_path, save_weights_only=True, save_freq='epoch', period=1)

94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
history = model.fit(
  train_ds,
  validation_data = val_ds,
  epochs=30,
  batch_size=16,
  callbacks = checkpoint_callback)


Epoch 1/30
10/10 [==============================] - 12s 802ms/step - loss: 91.2736 - accuracy: 0.5500 - val_loss: 48.5879 - val_accuracy: 0.4000
Epoch 2/30
10/10 [==============================] - 7s 692ms/step - loss: 73.1823 - accuracy: 0.5813 - val_loss: 72.0743 - val_accuracy: 0.3750
Epoch 3/30
10/10 [==============================] - 7s 687ms/step - loss: 44.0801 - accuracy: 0.6000 - val_loss: 18.7626 - val_accuracy: 0.5750
Epoch 4/30
10/10 [==============================] - 7s 699ms/step - loss: 22.8797 - accuracy: 0.6750 - val_loss: 8.9953 - val_accuracy: 0.7750
Epoch 5/30
10/10 [==============================] - 7s 703ms/step - loss: 24.3453 - accuracy: 0.6438 - val_loss: 12.4218 - val_accuracy: 0.5750
Epoch 6/30
10/10 [==============================] - 7s 710ms/step - loss: 14.2535 - accuracy: 0.7000 - val_loss: 7.5190 - val_accuracy: 0.7500
Epoch 7/30
10/10 [==============================] - 7s 703ms/step - loss: 10.5501 - accuracy: 0.7063 - val_loss: 10.5226 - val_accuracy: 

In [ ]:
from matplotlib import pyplot
import sys

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	pyplot.tight_layout()
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

summarize_diagnostics(history)

In [ ]:

val_ds = tf.keras.utils.image_dataset_from_directory(
  file_dir,
  image_size= (512, 640),
  label_mode = 'categorical',
  color_mode = "grayscale",
  batch_size=batch_size)

model.evaluate(val_ds)


Found 200 files belonging to 2 classes.
13/13 [==============================] - 1s 45ms/step - loss: 1.4372e-04 - accuracy: 1.0000


[0.00014371922588907182, 1.0]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# model.save('/content/drive/MyDrive/simpler.h5')
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 512, 640, 1)]     0         
                                                                 
 lambda_2 (Lambda)           (None, 512, 640, 3)       0         
                                                                 
 resnet101 (Functional)      (None, 16, 20, 2048)      42658176  
                                                                 
 flatten_2 (Flatten)         (None, 655360)            0         
                                                                 
 dense_8 (Dense)             (None, 256)               167772416 
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 128)               3289

In [ ]:
import shutil

# Define the source and destination paths
source_path = "/content/model_checkpoint_21.h5"
destination_path = "/content/drive/MyDrive/model_checkpoint_21.h5"

# Move the file
shutil.move(source_path, destination_path)

'/content/drive/MyDrive/model_checkpoint_21.h5'

In [ ]:
model.save('/content/drive/MyDrive/resnet101_set1.h5')
